In [1]:
import gym
from gym import spaces
import pandas as pd
import sys
sys.path.append(r"../../../")
#sys.path.append(r"../utils/data_manipulation")
from app_project_utils import align_multiple_measurement_times, get_target_variables, load_data
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier

C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
df = pd.read_csv(r"data\DeppClinic_patient_data.csv")
df = df.fillna(0)
time1_df = df[df.measurement.isin(['Time 1'])]
time2_df = df[df.measurement.isin(['Time 2'])]
time3_df = df[df.measurement.isin(['Time 3'])]
reshaped = time2_df.merge(time3_df, on='id', suffixes=['_time_2', '_time_3'])
reshaped = reshaped.merge(time1_df, on='id')
reshaped['Y'] = reshaped['modcon_target_time_2'].astype(bool) | reshaped['modcon_target_time_3'].astype(bool)

In [3]:
class DiagnosticEnv(gym.Env):
    def __init__(self, df, patient_id, questions, max_questions=8):
        super(DiagnosticEnv, self).__init__()

        self.df = df
        self.patient_id = patient_id
        self.max_questions = max_questions

        # Define action and observation space
        self.question_list = questions
        self.action_space = spaces.Discrete(len(self.question_list))

        # Assuming answers are discrete choices, modify as needed
        self.observation_space = spaces.Dict({question: spaces.Discrete(5) for question in self.question_list}) 
        self.state = None
        self.reset()

    def reset(self):
        self.state = {question: None for question in self.question_list}
        self.counter = 0
        return self.state

    def step(self, action):
        question = self.question_list[action]
        answer = self.get_value(question)
        self.state[question] = answer

        self.counter += 1
        is_done = self.counter >= self.max_questions
        reward = self.calculate_reward() if is_done else 0

        return self.state, reward, is_done

    def get_value(self, question):
        value = self.df.query("id == @self.patient_id")[question].iloc[0]
        return value

    def calculate_reward(self):
        def create_Y(df, state, row):
            X_cols = []
            for k, v in state.items():
                if v is not None:
                    X_cols.append(k)
            Y = df['Y']
            X = df[X_cols]
            x_test = row[X_cols]
            y_test = row['Y']

            return X, Y.astype(int), x_test, y_test
        
        row = self.df.query("id == @self.patient_id")
        
        X, Y, x_test, y_test = create_Y(self.df, self.state, row)
        clf = RandomForestClassifier()

        clf.fit(X,Y)
        y_pred = clf.predict(x_test)
        score = (y_pred == y_test).astype(int)
        return score.values[0]

In [4]:

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.95):
        self.q_table = dict()  # Dictionary to store Q-values
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor

    def choose_action(self, state):
        return self.action_space.sample()

    
    def learn(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        next_max = max(self.q_table.get(next_state, {}).values(), default=0)

        new_value = old_value + self.learning_rate * (reward + self.discount_factor * next_max - old_value)
        self.q_table.setdefault(state, {})[action] = new_value


In [5]:
from random import choices

In [7]:
choices(list(df.columns), k=16)


['mfq_14',
 'sci_af_ca_4',
 'sci_af_ca_35',
 'sci_af_ca_17',
 'c_ssrs_4_2weeks',
 'mfq_24',
 'sci_af_ca_28',
 'c_ssrs_5_2weeks',
 'c_ssrs_3_2weeks',
 'mfq_24',
 'sci_af_ca_23',
 'sci_af_ca_20',
 'sci_af_ca_26',
 'sci_af_ca_38',
 'c_ssrs_2_life',
 'sci_af_ca_14']

In [8]:
questions = ['mfq_14',
 'sci_af_ca_4',
 'sci_af_ca_35',
 'sci_af_ca_17',
 'c_ssrs_4_2weeks',
 'mfq_24',
 'sci_af_ca_28',
 'c_ssrs_5_2weeks',
 'c_ssrs_3_2weeks',
 'mfq_24',
 'sci_af_ca_23',
 'sci_af_ca_20',
 'sci_af_ca_26',
 'sci_af_ca_38',
 'c_ssrs_2_life',
 'sci_af_ca_14']


In [9]:
    # Set up environment (assuming env is your DiagnosticEnv instance)
    env = DiagnosticEnv(reshaped, patient_id='S6442', questions=questions, max_questions=6)  # fill with your parameters

    # Initialize agent
    agent = QLearningAgent(env.action_space)


In [ ]:
for patient_id in reshaped.id[:30]:

    total_episodes = 100
    # Set up environment (assuming env is your DiagnosticEnv instance)
    env = DiagnosticEnv(reshaped, patient_id=patient_id, questions=questions, max_questions=6)  # fill with your parameters

    # Initialize agent
    agent = QLearningAgent(env.action_space)

    # Training loop
    for episode in range(total_episodes):
        state = env.reset()
        done = False

        while not done:
            action = agent.choose_action(str(state))  # State converted to string for simplicity
            print(f"{action = }")
            next_state, reward, done = env.step(action)
            agent.learn(str(state), action, reward, str(next_state))
            state = next_state

action = 5
action = 9
action = 8
action = 12
action = 10
action = 13


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 14
action = 5
action = 11
action = 9
action = 12
action = 8


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 8
action = 7
action = 6
action = 9
action = 1
action = 5


...

action = 10
action = 14
action = 12
action = 10
action = 8
action = 9


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


action = 14
action = 1
action = 14
action = 9
action = 0
action = 3


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 9
action = 9
action = 12
action = 15
action = 8
action = 15
